In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np
#np.set_printoptions(threshold=np.inf)

import scipy.sparse as spr
import os

In [2]:
nbX = 10
nbT = 40
nbY = 2

In [3]:
IdX = spr.identity(nbX)
LX = spr.dia_matrix(np.roll(IdX.toarray(),1,axis=1))
RX = spr.coo_matrix(np.concatenate((np.ones((nbX,1)),np.zeros((nbX,nbX-1))), axis =1))
P = spr.kron(np.array([[1,0]]).T, (0.75 * IdX + 0.25 * LX)) + spr.kron(np.array([[0,1]]).T, RX)
P.toarray()

array([[0.75, 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.75, 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.75, 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.75, 0.25, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.75, 0.25, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25],
       [0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.

In [4]:
IdT = spr.identity(nbT)
NT_temp = np.roll(IdT.toarray(),1,axis=1)
NT_temp[nbT-1]=0
NT = spr.dia_matrix(NT_temp)
A = spr.kron(spr.kron(IdT, np.array([[1,1]]).T), IdX) - spr.kron(NT, P)
A.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [5]:
overhaulCost = 8000
maintCost = lambda x: x * 500
beta = 0.9
discount = lambda x: beta**x

In [6]:
n1_x = np.repeat(1, nbX)
u_xy = np.concatenate((-np.asarray(list(map(maintCost,range(1,nbX)))), np.repeat(-overhaulCost, nbX+1)), axis = None)
u_xyt = np.kron(np.asarray(list(map(discount,range(1,nbT+1)))),u_xy)

b_xt = np.concatenate((n1_x, np.repeat(0,int(nbX*(nbT-1)))),axis =None)

In [7]:
b_xt

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [58]:
m = grb.Model('BusMaintenance')
x = m.addMVar(shape=len(b_xt), name="x", lb=-grb.GRB.INFINITY)
m.setObjective(b_xt.T @ x, grb.GRB.MINIMIZE)
rhs = u_xyt.T
m.addConstr(A @ x >= rhs, name="Constr")
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 800 rows, 400 columns and 1970 nonzeros
Model fingerprint: 0xd28184f1
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 7e+03]
Presolve removed 800 rows and 400 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5786600e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -1.578660020e+05


In [66]:
if m.status == grb.GRB.Status.OPTIMAL:
    solution = x.X
    for state in range(nbX):
        print(solution[state])

-10725.480069535954
-13593.2845123476
-15735.668876280946
-16830.22408011453
-16830.22408011453
-16830.22408011453
-16830.22408011453
-16830.22408011453
-16830.22408011453
-16830.22408011453


## Backward Induction

In [120]:
U_x_t = np.zeros((nbX,nbT))
u_xyt_reshape = u_xyt.reshape(nbX, nbY, nbT, order = 'F') #note the order is Fortran order
contVals = u_xyt_reshape [:,:,nbT - 1].max(axis = 1) 
U_x_t[:,nbT - 1] = contVals

for t in range(nbT-2, -1, -1):
    myopic = u_xyt_reshape[:,:,t]
    EcontVals = (P @ contVals).reshape([nbX, nbY], order = 'F')
    contVals = myopic + EcontVals
    contVals = contVals.max(axis = 1)
    U_x_t[:, t] = contVals

print('\n')
print(U_x_t[:, 0])



[-10725.48006954 -13593.28451235 -15735.66887628 -16830.22408011
 -16830.22408011 -16830.22408011 -16830.22408011 -16830.22408011
 -16830.22408011 -16830.22408011]
